In [23]:
import numpy as np
import pyccl as ccl
import scipy.integrate

## Comoving angular-diameter distance 

$d_C = d_{H}\int_{0}^{z} \frac{1}{E(z')} dz'$

$E(z)= \sqrt{\Omega_{r,0} (1+z)^4 + \Omega_{m,0} (1+z)^3 + \Omega_{\Lambda,0} + (1- \Omega_{0})(1+z)^2}$

Generalization to curvature:

$ D_M = \begin{cases} 
      \frac{d_H}{\sqrt{\Omega_k}} \sinh{\left( \frac{\sqrt{\Omega_k} d_c}{d_H} \right)} & \Omega_k > 0 \\
      d_c & \Omega_k = 0 \\
      \frac{d_H}{\sqrt{|\Omega_k|}} \sin{\left( \frac{\sqrt{|\Omega_k|} d_c}{d_H} \right)} & \Omega_k < 0 
   \end{cases}
$

+ $d_{H}$  = Hubble Distance
+ $\Omega_{r,0}$ = Omega Radiation
+ $\Omega_{m,0}$ = Omega Matter
+ $\Omega_{\Lambda,0}$ = Energy density of Dark Energy
+ $\Omega_{0}$  = Omega Zero
+ $\Omega_{k}$  = Curvature of the Universe

In [24]:
#Physics Variables
G= 7e-11
c= 2.998e8
H0 = (6.8e-18/3.0857)
hubble_distance= c/H0
#Cosmology Variables
omega_matter=0.27
omega_lambda=0.73
omega_radiation= 1 - omega_lambda - omega_matter
omega_0= omega_matter + omega_radiation + omega_lambda
omega_kappa = 1. - omega_0

In [25]:
def E_parameter(z):
    return (np.sqrt(omega_radiation*(1+z)**4 + omega_matter*(1+z)**3 
                    + omega_lambda + (1-omega_0)*(1+z)**2))

In [26]:
def comovingDistance(z1, z2):
        if np.isclose(omega_kappa, 0.):
            return hubble_distance * scipy.integrate.quad(
                    lambda z: 1/E_parameter(z),z1, z2)[0]
        elif omega_kappa > 0.:
            return hubble_distance \
                * np.sinh(np.sqrt(omega_kappa) * scipy.integrate.quad(
                lambda z: 1/E_parameter(z), z1, z2)[0]) / np.sqrt(omega_kappa)
        else:
            return hubble_distance \
                *np.sin(np.sqrt(np.fabs(omega_kappa))*scipy.integrate.quad(
                lambda z: 1/E_parameter(z),z1,z2)[0])/np.sqrt(np.fabs(omega_kappa))

In [27]:
#https://github.com/LSSTDESC/CCL/blob/master/examples/Distance%20Calculations%20Example.ipynb
comovingDistance(0,0.5)/(3.09e22)

1962.1392803585286

## Angular diameter distances:
$
d_{LS}= \frac{D_{LS}}{1+z_S}  \quad
d_{L}= \frac{D_{L}}{1+z_L}  \quad
d_{S}= \frac{D_{S}}{1+z_s}
$

## Einstein Radius:

$\theta = \sqrt{\frac{4GM}{c^2} \frac{d_{LS}}{d_L d_s}}$

I am going to use the values from the HSC double source place lens system (Eye of Horus) as an [example](https://arxiv.org/pdf/1606.09363.pdf). 

In [28]:
#System Variables:
M = 140.0e41 # Total mass including DM. Estelar mass: 14.0e41
Zlens = 0.795
Zs1 = 1.302
Zs2 = 1.988

In [29]:
#Comoving Distances:
dlens = comovingDistance(0,Zlens)/(1+Zlens)
ds1 = comovingDistance(0,Zs1)/(1+Zs1) 
ds2 = comovingDistance(0,Zs2)/(1+Zs2)
dls1 = comovingDistance(Zlens,Zs1)/(1+Zs1)
dls2 = comovingDistance(Zlens,Zs2)/(1+Zs2)

In [30]:
def Einstein_Radius(DL, DS, DLS):
    return np.sqrt((4*G*M*DLS)/(c**2 * DL * DS))

In [31]:
print("Inner Einstein radius: " + str(206265*Einstein_Radius(dlens, ds1, dls1)) + "''"
      "\n"+"Outer Einstein radius: "+str(206265*Einstein_Radius(dlens, ds2, dls2)) + "''" )

Inner Einstein radius: 3.380397135693867''
Outer Einstein radius: 4.17632754416823''


The value of the outer radius found in the paper was 3''.3. The difference between the calculated value here and the measured value could be due to the fact that we used the point approximation for the lens but in this case the lens is an extented object.